In [1]:
import tkinter as tk
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk

class tkApp:
    def __init__(self):
        #---------------initialize frame-------------
        self.root=tk.Tk()
        self.root.geometry("600x400")
        self.root.title("SIR Model for Infectious Diseases")
        self.root.update()

        self.left_frame=tk.Frame(master=self.root)
        self.right_frame=tk.Frame(master=self.root)
        
        self.plot_label=tk.Label(master=self.right_frame,
                                 text='Solved SIR Model')
        self.plot_label.pack()

        self.widget=None
        self.toolbar=None

        #----------------4th order Runge Kutta method for solving ODEs---------
        def solveSIR(self):
            if self.widget:
                  self.widget.destroy()
            if self.toolbar:
                  self.toolbar.destroy()
            i_str=iinitial.get()
            i0=float(i_str)
            s_str=sinitial.get()
            s0=float(s_str)
            r0=1-i0-s0
            beta_str=b.get()
            beta=float(beta_str)
            gamma_str=g.get()
            gamma=float(gamma_str)
            n=100
            dt=0.5

            # susceptible ode
            def dsdt(t,s,i):
                return(-beta*s*i)

            # infected ode
            def didt(t,s,i):
                return(beta*s*i-gamma*i)

            #basic SIR model 
            def RK4SIR(n,beta, gamma,s0,i0,r0,dt):
                #initialize the arrays for solutions
                S=[s0]+[0]*n
                I=[i0]+[0]*n
                R=[r0]+[0]*n
            
                #calculate each step using RK4
                for i in range(0,n):
                    Si=S[i]
                    Ii=I[i]
                

                    sk1=dsdt(i,Si,Ii)
                    ik1=didt(i,Si,Ii)
        

                    sk2=dsdt(i+dt/2,Si+dt/2*sk1,Ii+dt/2*ik1)
                    ik2=didt(i+dt/2,Si+dt/2*sk1,Ii+dt/2*ik1)

                    sk3=dsdt(i+dt/2,Si+dt/2*sk2,Ii+dt/2*ik2)
                    ik3=didt(i+dt/2,Si+dt/2*sk2,Ii+dt/2*ik2)

                    sk4=dsdt(i+dt,Si+dt*sk3,Ii+dt*ik3)
                    ik4=didt(i+dt,Si+dt*sk3,Ii+dt*ik3)

                    S[i+1]=Si+dt/6*(sk1+2*sk2+2*sk3+sk4)
                    I[i+1]=Ii+dt/6*(ik1+2*ik2+2*ik3+ik4)

                #caculate recovered array
                for i in range(len(R)):
                    R[i]=1-I[i]-S[i]

                return S,I,R

            #solve basic SIR model and plot    
            s,i,r=RK4SIR(n,beta,gamma,s0,i0,r0,dt)
            fig=Figure(figsize=(5,5), dpi=100)
            plot1=fig.add_subplot(111)
            plot1.clear()

            t=list(range(0,n+1))
            plot1.plot(t,s,'r')
            plot1.plot(t,i,'b')
            plot1.plot(t,r,'g')
            
            canvas=FigureCanvasTkAgg(fig, master=self.right_frame)
            canvas.draw()

            self.toolbar=NavigationToolbar2Tk(canvas, self.right_frame)
            self.toolbar.update()

            self.widget=canvas.get_tk_widget()
            self.widget.pack()
        
            
        
        #----------initialize variables----------
        b=tk.StringVar()
        g=tk.StringVar()
        sinitial=tk.StringVar()
        iinitial=tk.StringVar()

        #-------------left frame sub-frames--------
        self.beta_frame=tk.Frame(master=self.left_frame)
        self.gamma_frame=tk.Frame(master=self.left_frame)
        self.S_frame=tk.Frame(master=self.left_frame)
        self.I_frame=tk.Frame(master=self.left_frame)
        self.button_frame=tk.Frame(master=self.left_frame)

        #beta frame
        self.beta_label=tk.Label(master=self.beta_frame, 
                                 text='beta')
        self.beta_slider=tk.Scale(master=self.beta_frame, 
                                  from_=0,
                                  to=2, 
                                  resolution=0.1,
                                  orient=tk.HORIZONTAL, 
                                  variable=b)
        self.beta_label.pack(side='left')
        self.beta_slider.pack(side='left')

        #gamma frame
        self.gamma_label=tk.Label(self.gamma_frame, 
                                  text='gamma')
        self.gamma_slider=tk.Scale(self.gamma_frame, 
                                   from_=0, 
                                   to=1, 
                                   resolution=0.1, 
                                   orient=tk.HORIZONTAL, 
                                   variable=g)
        self.gamma_label.pack(side='left')
        self.gamma_slider.pack(side='left')

        #S0 frame
        self.S_label=tk.Label(self.S_frame, 
                              text='S0')
        self.S_slider=tk.Scale(self.S_frame, 
                               from_=0, 
                               to=1, 
                               resolution=0.01, 
                               orient=tk.HORIZONTAL, 
                               variable=sinitial)
        self.S_label.pack(side='left')
        self.S_slider.pack(side='left')

         #I0 frame
        self.I_label=tk.Label(self.I_frame, 
                              text='I0')
        self.I_slider=tk.Scale(self.I_frame, 
                               from_=0, 
                               to=1, 
                               resolution=0.01, 
                               orient=tk.HORIZONTAL, 
                               variable=iinitial)
        self.I_label.pack(side='left')
        self.I_slider.pack(side='left')

        
        #button widgets
        self.calc_button=tk.Button(self.button_frame, 
                                   text='Calculate', 
                                   command=lambda: [solveSIR(self)], 
                                   height=2, 
                                   width=10)
        self.quit_button=tk.Button(self.button_frame, 
                                   text='Quit', 
                                   command=self.root.destroy)
        self.calc_button.pack(side='left')
        self.quit_button.pack(side='left')

        #--------pack frames----------
        self.beta_frame.pack(side='top')
        self.gamma_frame.pack(side='top')
        self.S_frame.pack(side='top')
        self.I_frame.pack(side='top')
        self.button_frame.pack(side='top')

        self.left_frame.pack(side='left')
        self.right_frame.pack(side='left')



        self.root.mainloop()

#-----------call class to run GUI-----------
if __name__=='__main__':
    TK_Window=tkApp()